# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
df = pd.read_csv("../output_data/cities.csv")
df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,east london,-33.0153,27.9116,55.40,100,40,6.91,ZA,1620971686
1,hithadhoo,-0.6000,73.0833,82.02,77,89,8.79,MV,1620971686
2,ushuaia,-54.8000,-68.3000,39.20,87,0,5.50,AR,1620971686
3,prince rupert,54.3161,-130.3201,46.40,76,90,4.61,CA,1620971687
4,ayacucho,-13.1583,-74.2239,50.67,88,27,0.74,PE,1620971687


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps.
gmaps.configure(api_key = g_key)

In [4]:
# Use the Lat and Lng as locations and Humidity as the weight.
location = df[["Lat", "Lng"]]
weight = df["Humidity"]

fig = gmaps.figure()

# Add Heatmap layer to map.
heatmap_layer = gmaps.heatmap_layer(location, 
                                    weights = weight,
                                    dissipating = False, 
                                    max_intensity = 100,
                                    point_radius = 3)
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
weather_df = df.loc[(df["Max Temp"] < 80) &
                    (df["Max Temp"] > 70) &
                    (df["Wind Speed"] < 10) &
                    (df["Cloudiness"] == 0), :]
weather_df = weather_df.dropna()
weather_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
183,praya,14.9215,-23.5087,75.20,50,0,8.05,CV,1620971634
219,yuci,37.6803,112.7319,75.20,43,0,6.71,CN,1620971771
259,komsomolskiy,40.4272,71.7189,77.00,25,0,9.22,UZ,1620971787
286,inzer,54.2167,57.5556,79.61,36,0,1.66,RU,1620971797
297,petropavl,54.8753,69.1628,71.60,33,0,6.71,KZ,1620971801
324,abu samrah,35.3029,37.1841,73.76,25,0,5.86,SY,1620971812
335,athens,37.9795,23.7162,71.01,45,0,8.05,GR,1620971614
389,shubarkuduk,49.1447,56.4872,79.41,22,0,8.34,KZ,1620971838
391,bakal,54.9417,58.8083,77.00,31,0,2.73,RU,1620971839
434,agdas,40.6500,47.4761,75.20,64,0,3.44,AZ,1620971856


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [10]:
hotel_df = weather_df
hotel_df["Hotel Name"] = ""
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
183,praya,14.9215,-23.5087,75.20,50,0,8.05,CV,1620971634,
219,yuci,37.6803,112.7319,75.20,43,0,6.71,CN,1620971771,
259,komsomolskiy,40.4272,71.7189,77.00,25,0,9.22,UZ,1620971787,
286,inzer,54.2167,57.5556,79.61,36,0,1.66,RU,1620971797,
297,petropavl,54.8753,69.1628,71.60,33,0,6.71,KZ,1620971801,
324,abu samrah,35.3029,37.1841,73.76,25,0,5.86,SY,1620971812,
335,athens,37.9795,23.7162,71.01,45,0,8.05,GR,1620971614,
389,shubarkuduk,49.1447,56.4872,79.41,22,0,8.34,KZ,1620971838,
391,bakal,54.9417,58.8083,77.00,31,0,2.73,RU,1620971839,
434,agdas,40.6500,47.4761,75.20,64,0,3.44,AZ,1620971856,


In [11]:
for index, row in hotel_df.iterrows():
    try:
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
        params = {"keyword": "hotel",
                  "radius": 5000,
                  "key": g_key,}

        lat = row['Lat']
        lng = row['Lng']
        params['location'] = f"{lat}, {lng}"
        hotel_data = requests.get(base_url, params=params).json()
        hotel_df.loc[index, "Hotel Name"] = hotel_data["results"][0]["name"]
        
    except IndexError:
        hotel_df.loc[index, "Hotel Name"] = "NaN"
        
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
183,praya,14.9215,-23.5087,75.20,50,0,8.05,CV,1620971634,Hotel Oasis Atlantico Praiamar
219,yuci,37.6803,112.7319,75.20,43,0,6.71,CN,1620971771,Eden International Hotel
259,komsomolskiy,40.4272,71.7189,77.00,25,0,9.22,UZ,1620971787,Diyor hotel
286,inzer,54.2167,57.5556,79.61,36,0,1.66,RU,1620971797,NaN
297,petropavl,54.8753,69.1628,71.60,33,0,6.71,KZ,1620971801,TELEGRAPH INN
324,abu samrah,35.3029,37.1841,73.76,25,0,5.86,SY,1620971812,NaN
335,athens,37.9795,23.7162,71.01,45,0,8.05,GR,1620971614,"Hotel Grande Bretagne, a Luxury Collection Hot..."
389,shubarkuduk,49.1447,56.4872,79.41,22,0,8.34,KZ,1620971838,NaN
391,bakal,54.9417,58.8083,77.00,31,0,2.73,RU,1620971839,NaN
434,agdas,40.6500,47.4761,75.20,64,0,3.44,AZ,1620971856,AZPETROL YDM


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [9]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))